In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

import os

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
data = {i[:8]:pd.read_csv("clean/" + i, sep="|", skiprows=[0, 1]) 
        for c, i in enumerate(os.listdir("clean"))}

In [ ]:
full = []
for i in data:
    df = data[i]
    df.drop(["notes", "sources"], axis=1, inplace=True)
    df.columns = ["place", "confirmed_cases", "deaths"]
    df["date"] = i
    df.loc[df.place == "CHINA TOTAL", "confirmed_cases"] = df.confirmed_cases[:-1].astype(int).sum()
    df.loc[df.place == "CHINA TOTAL", "deaths"] = df.deaths[:-1].astype(int).sum()
    df["confirmed_cases"] = df.confirmed_cases.astype(int)
    df["deaths"] = df.deaths.astype(int)
    full.append(data[i])
full = pd.concat(full)
full.reset_index(drop=True, inplace=True)

In [ ]:
t = []
for d in ("20200122", "20200123", "20200124"):
    confirmed_cases = full[full.date == d].confirmed_cases.sum()
    deaths = full[full.date == d].deaths.sum()
    t.append(["CHINA TOTAL", confirmed_cases, deaths, d])
t = pd.DataFrame(t)
t.columns = ["place", "confirmed_cases", "deaths", "date"]
full = pd.concat([full, t])
full.reset_index(drop=True, inplace=True)

In [ ]:
china_total = full[full.place == "CHINA TOTAL"]
hubei = full[full.place == "Hubei"]

In [ ]:
plt.figure(figsize=(15, 20))

plt.subplot(2, 1, 1)
sns.lineplot(x="date", y="confirmed_cases", data=china_total, label="China total")
sns.lineplot(x="date", y="confirmed_cases", data=hubei, label="Hubei province")
plt.ylim(0, 48000)
plt.xticks(rotation=90)
plt.title("Number of confirmed cases in China and Hubei province")

plt.subplot(2, 1, 2)
sns.lineplot(x="date", y="deaths", data=china_total, label="China total")
sns.lineplot(x="date", y="deaths", data=hubei, label="Hubei province")
plt.ylim(0, 1200)
plt.xticks(rotation=90)
plt.title("Number of deaths in China and Hubei province")

plt.legend()
plt.show()

In [ ]:
current = full[(full.place != "CHINA TOTAL") & (full.date == "20200212")]
current.columns = ["name", "confirmed_cases", "deaths", "date"]

In [ ]:
china = gpd.read_file("china.json")

In [ ]:
pinyin = ["Xinjiang", "Xizang", "Inner Mongolia", "Qinghai", "Sichuan", "Heilongjiang", "Gansu", "Yunnan", "Guangxi", 
          "Hunan", "Shaanxi", "Guangdong", "Jilin", "Hebei", "Hubei", "Guizhou", "Shandong", "Jiangxi", "Henan", 
          "Liaoning", "Shanxi", "Anhui", "Fujian", "Zhejiang", "Jiangsu", "Chongqing", "Ningxia", "Hainan", "Taiwan", 
          "Beijing", "Tianjin", "Shanghai", "Hong Kong", "Macau"]
china["name"] = pinyin

In [ ]:
current = pd.merge(china, current, how="left")
current.loc[1, "confirmed_cases"] = 0
current.loc[1, "deaths"] = 0
current.loc[1, "date"] = "20200212"

# current.loc[14, "confirmed_cases"] = 0

In [ ]:
current

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 10))
current.plot(column="confirmed_cases", cmap="Reds", linewidth=0.8, edgecolor='0.8', ax=ax)
ax.axis("off")

sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=0, vmax=33366))
cbar = fig.colorbar(sm)
plt.show()